### A Practical Introduction to Integer Programming
#### Igor Ferst, PyOhio 2019

Code based on:
https://github.com/google/or-tools/blob/stable/ortools/linear_solver/samples/simple_mip_program.py

Install `ortools` if you haven't already.

In [ ]:
import sys
!{sys.executable} -m pip install --user ortools

Instantiate a solver.

In [3]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver('PyOhio 2019', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

Define all variables, one for each vehicle/route pair.

In [4]:
x = {
    'A1': solver.IntVar(0.0, 1.0, 'A1'),
    'A2': solver.IntVar(0.0, 1.0, 'A2'),
    'A3': solver.IntVar(0.0, 1.0, 'A3'),
    'A4': solver.IntVar(0.0, 1.0, 'A4'),
    'B1': solver.IntVar(0.0, 1.0, 'B1'),
    'B2': solver.IntVar(0.0, 1.0, 'B2'),
    'B3': solver.IntVar(0.0, 1.0, 'B3'),
    'B4': solver.IntVar(0.0, 1.0, 'B4'),    
    'C1': solver.IntVar(0.0, 1.0, 'C1'),
    'C2': solver.IntVar(0.0, 1.0, 'C2'),
    'C3': solver.IntVar(0.0, 1.0, 'C3'),
    'C4': solver.IntVar(0.0, 1.0, 'C4'),    
}

print(('Total number of variables: %d' % solver.NumVariables()))

Total number of variables: 12


Define all constraints.

In [5]:
# Coverage constraints.

solver.Add( x['A1'] + x['B1'] + x['C1'] == 1 )
solver.Add( x['A2'] + x['B2'] + x['C2'] == 1 )
solver.Add( x['A3'] + x['B3'] + x['C3'] == 1 )
solver.Add( x['A4'] + x['B4'] + x['C4'] == 1 )

# Capacity constraints.

solver.Add( x['A1'] == 0 )
solver.Add( x['A2'] == 0 )
solver.Add( x['B1'] == 0 )

# Overlap constraints.

solver.Add( x['A1'] + x['A2'] <= 1 ) 
solver.Add( x['B1'] + x['B2'] <= 1 ) 
solver.Add( x['C1'] + x['C2'] <= 1 ) 

solver.Add( x['A3'] + x['A4'] <= 1 ) 
solver.Add( x['B3'] + x['B4'] <= 1 ) 
solver.Add( x['C3'] + x['C4'] <= 1 ) 

print(('Total number of constraints: %d' % solver.NumConstraints()))

Total number of constraints: 13


Define function to be minimized (objective function).

In [6]:
solver.Minimize(
    4 * x['A1'] + 6 * x['B1'] + 10 * x['C1'] + 
    4 * x['A2'] + 6 * x['B2'] + 10 * x['C2'] + 
    4 * x['A3'] + 6 * x['B3'] + 10 * x['C3'] + 
    4 * x['A4'] + 6 * x['B4'] + 10 * x['C4']    
)

Now we're ready to solve!

In [7]:
result_status = solver.Solve()

# Confirm the problem has an optimal solution.
assert result_status == pywraplp.Solver.OPTIMAL

# Confirm the solution looks legit (when using solvers others than
# GLOP_LINEAR_PROGRAMMING, verifying the solution is highly recommended).
assert solver.VerifySolution(1e-7, True)

# Get vehicle assignments.
assignments = [x_var for x_var in x.values() if x_var.solution_value() == 1.0]
print('Assignments: %s' % str(assignments))

# The objective value of the solution (i.e. minimal value of the function to be minimized)
print(('Minimal objective value: %f' % solver.Objective().Value()))

Assignments: [A4, B2, B3, C1]
Minimal objective value: 26.000000
